## To learn more about AML pipelines and remote jobs please refer to https://github.com/Azure/azureml-examples

## This notebook was test using Python 3.8 - AzureML kernel

In [107]:
%pip install azure-ai-ml

     |████████████████████████████████| 356 kB 55.7 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 4.6 MB/s  eta 0:00:01
ERROR: fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.
ERROR: pycaret 2.3.10 has requirement pyyaml<6.0.0, but you'll have pyyaml 6.0 which is incompatible.
ERROR: pycaret 2.3.10 has requirement scikit-learn==0.23.2, but you'll have scikit-learn 0.22.1 which is incompatible.
ERROR: pandas-profiling 3.2.0 has requirement joblib~=1.1.0, but you'll have joblib 0.14.1 which is incompatible.
ERROR: pandas-profiling 3.2.0 has requirement markupsafe~=2.1.1, but you'll have markupsafe 2.0.1 which is incompatible.
ERROR: datasets 1.8.0 has requirement tqdm<4.50.0,>=4.27, but you'll have tqdm 4.63.0 which is incompatible.
ERROR: azureml-widgets 1.42.0 has requirement jinja2<=2.11.2, but you'll have jinja2 2.11.3 which is incompatible.
ERROR: azureml-contrib-notebook 1.42.0 has requirement nbconvert<6, but you'll have nbconvert 6.5.0 which is 

  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.12.0
    Uninstalling azure-storage-blob-12.12.0:
      Successfully uninstalled azure-storage-blob-12.12.0
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
Note: you may need to restart the kernel to use updated packages.


## Import required libraries

In [28]:
# import required libraries
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    Model,
    Environment,
    BatchRetrySettings,
)
# from azure.ai.ml.constants import AssetType
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import BatchDeploymentOutputAction
import mlflow
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model

### Configure workspace details and get a handle to the workspace

In [29]:
# Enter details of your AML workspace
subscription_id = "xxxxxxxxx"
resource_group = "aml-v2-book"
workspace = "aml2-ws"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

### Create a batch endpoint for inferencing

In [30]:
import datetime

batch_endpoint_name = "titanic-batchendpoint-sdk"

# create a batch endpoint
endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="this is a sample batch endpoint created using sdk",
    tags={"tag1": "val1"},
)

In [31]:
ml_client.begin_create_or_update(endpoint)

BatchEndpoint({'scoring_uri': 'https://titanic-batchendpoint-sdk2.eastus.inference.ml.azure.com/jobs', 'swagger_uri': None, 'provisioning_state': 'Succeeded', 'name': 'titanic-batchendpoint-sdk2', 'description': 'this is a sample batch endpoint created using sdk', 'tags': {'tag1': 'val1'}, 'properties': {'BatchEndpointCreationApiVersion': '2022-05-01', 'azureml.onlineendpointid': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourcegroups/aml-v2-book/providers/microsoft.machinelearningservices/workspaces/aml2-ws/onlineendpoints/titanic-batchendpoint-sdk2', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/be:8fee5b67-b42b-442e-b2a1-5aa4a497553f:35200320-1835-45b1-98b0-223b42f76ce2?api-version=2022-05-01'}, 'id': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspac

### Retrieving an existing registered model from the workspace

In [32]:
model = ml_client.models.get("titanic_survival_model_", label="latest")

### Creating a batch deployment 

In [33]:
# create a batch deployment
deployment = BatchDeployment(
    name="titanic-deployment-sdk",
    description="titanic deployment created using sdk",
    endpoint_name="titanic-batchendpoint-sdk2",
    model=model,
    compute="cpu-cluster",
    instance_count=1,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
    logging_level="info",
)

In [34]:
ml_client.begin_create_or_update(deployment)

BatchDeployment({'endpoint_name': 'titanic-batchendpoint-sdk2', 'type': None, 'name': 'titanic-deployment-sdk', 'description': 'titanic deployment created using sdk', 'tags': {}, 'properties': {}, 'id': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/batchEndpoints/titanic-batchendpoint-sdk2/deployments/titanic-deployment-sdk', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f3dde513490>, 'model': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/models/titanic_survival_model_/versions/3', 'code_configuration': None, 'environment': None, 'environment_variables': {}, 'compute': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/computes/cpu-cluster', 'resou

### Invoke your batch endpoint with a registered dataset

In [35]:
data = ml_client.data.get("titanic-test-data", version=7)

job = ml_client.batch_endpoints.invoke(
    endpoint_name="titanic-batchendpoint-sdk",
    input=Input(path=data.id, type="uri_file"), 
    deployment_name="titanic-deployment-sdk",  # name is required if default deployment is not set
    params_override=[{"mini_batch_size": "1"}, {"compute.instance_count": "1"}],
)


In [26]:
data.id

'/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/data/titanic-test-data/versions/7'

###  Monitor the batch scoring job

In [36]:
# get the details of the job
job_name = job.name
batch_job = ml_client.jobs.get(name=job_name)
print(batch_job.status)
# stream the job logs
ml_client.jobs.stream(name=job_name)

Running
RunId: 112a52ae-2615-43b0-b86e-8e8c3cdbe4b2
Web View: https://ml.azure.com/runs/112a52ae-2615-43b0-b86e-8e8c3cdbe4b2?wsid=/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourcegroups/aml-v2-book/workspaces/aml2-ws

Streaming logs/azureml/executionlogs.txt

[2022-12-04 19:11:48Z] Submitting 1 runs, first five are: e1ec955d:df43d50e-f325-43ed-b4d2-69b559a31ece
[2022-12-04 19:21:27Z] Completing processing run id df43d50e-f325-43ed-b4d2-69b559a31ece.

Execution Summary
RunId: 112a52ae-2615-43b0-b86e-8e8c3cdbe4b2
Web View: https://ml.azure.com/runs/112a52ae-2615-43b0-b86e-8e8c3cdbe4b2?wsid=/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourcegroups/aml-v2-book/workspaces/aml2-ws

